[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Humboldt-WI/bads/blob/master/algorithms_from_scratch/decision_trees.ipynb)

# Decision Trees

Decision trees are useful and powerful algorithms for classifying and regressing data. They work by on recursive partitioning which will be shown algorithmically in this notebook. Though most of the the time, sklearn can be used to implement this machine learning method, it is useful to take a look at the inner workings of tree-based algorithms. The following parts will first introduce a decision tree from scratch.

## Load modules and data

We will be using the HMEQ dataset from the [fifth demo notebook](https://github.com/Humboldt-WI/bads/blob/master/demo_notebooks/5_nb_supervised_learning.ipynb).

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_url = 'https://raw.githubusercontent.com/Humboldt-WI/bads/master/data/hmeq_modeling.csv' 

In [42]:
df = pd.read_csv(data_url, header = 0, index_col = 0) # import file with header and index as first row and column respectively

df.head() #inspect data to make sure it looks correct

,BAD,LOAN,MORTDUE,VALUE,YOJ,CLAGE,NINQ,CLNO,DEBTINC,DEROGzero,REASON_HomeImp,REASON_IsMissing,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self,DELINQcat_1,DELINQcat_1+
index,,,,,,,,,,,,,,,,,,,
0,True,-1.832283,-1.295882,-1.335526,0.266788,-1.075278,-0.065054,-1.297476,0.137456,True,1,0,0,1,0,0,0,0,0
1,True,-1.810666,-0.013474,-0.672699,-0.236615,-0.723092,-0.826792,-0.756608,0.137456,True,1,0,0,1,0,0,0,0,1
2,True,-1.789048,-1.654549,-1.839275,-0.668103,-0.368769,-0.065054,-1.189302,0.137456,True,1,0,0,1,0,0,0,0,0
3,True,-1.789048,-0.159552,-0.202559,-0.236615,-0.061033,-0.065054,-0.107566,0.137456,True,0,1,0,1,0,0,0,0,0
4,False,-1.767431,0.791699,0.311107,-0.811933,-1.088528,-0.826792,-0.756608,0.137456,True,1,0,1,0,0,0,0,0,0


## Decision tree from scratch

The following code is based on the material originally created by Sebastian Mantey for the Iris Dataset whose GitHub is here: https://github.com/SebastianMantey/Decision-Tree-from-Scratch. You can also search his video on YouTube. The code has been adapted to this course's format and fit to HMEQ Dataset.

Just in case you made changes to the data by running the above code, we first reload the data to a fixed and clear-defined starting point for our tree classifier. Recall that we defined the variable `data_url` above.

In [43]:
X = df.drop(['BAD'], axis=1) #code the variables in the most standard way for your usage
y = df[['BAD']]

In [44]:
print(type(X), type(y), X.shape, y.shape) # double check that types and dimensions are correct before proceeding

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> (5960, 18) (5960, 1)


### Helper Functions for the Tree

Decision trees work by recursively partitioning data. Many potential ways to split the data are calculated (eg. we are going to go through all unique values of each column and finding the midpoint between sequential values). For each potential split, we evaluate whether the target variable has more homogeneity in each leaf. This is done by calculating the 'impurity' of the parent node and comparing it with the sum of the impurity of the child nodes. There are three major impurity functions: entropy, gini and misclassification. We will be using entropy in our example. The split which yields the lowest impurity is chosen and the process is repeated for the new nodes (i.e.,recursion). 

The method of choosing the split that yields the lowest impurity is called the greedy search method. The following functions will help the decision tree implement greedy search tactics on the data. The algorithm stops either when purity in each node is reached or when it has reached a maximum depth (max amount of recursions we allow) specified in our function.

Many functions for the tree are not found in Python packages and it is cleaner to write them out first then put them together in our main algorithm. Each function below does a specific action, which will be used in our final tree function at the end.

In [45]:
def check_purity(y):
    
    'checks if a leaf node is perfectly pure, in other words, if the leaf node contains only one class'
    
    unique_classes = np.unique(y) #count number of classes in section of data

    if len(unique_classes) == 1: #check if the node is pure
        return True
    else:
        return False

In [46]:
def classify_data(y):
    
    'classifies data according to the majority class of each leaf'
    
    unique_classes, counts_unique_classes = np.unique(y, return_counts=True)
    #returns classes and no. of obs per class

    index = counts_unique_classes.argmax() #index of class with most obs
    classification = unique_classes[index] #class chosen for classification which is class with most obs
    
    prob = len(y[y == 1]) / len(y) #returns proportion of class 1 in the leaf (prob of class 1)

    return [classification, prob]

In [47]:
def get_potential_splits(X):
    
    'first, takes every unique value of every feature in the feature space, then finds the midpoint between each value'
    
    potential_splits = {}
    _, n_columns = X.shape #don't need rows, we choose the column to split on
    # only need second value of .shape which is columns
    
    for column_index in range(n_columns):
        potential_splits[column_index] = [] 
        values = X[:, column_index] 
        unique_values = np.unique(values) #get all unique values in each column

        for index in range(len(unique_values)): #all unique feature values
            if index != 0: #skip first value, we need the difference between next values
                current_value = unique_values[index]
                previous_value = unique_values[index - 1] #find a value and the next smallest value
                potential_split = (current_value + previous_value) / 2 #find difference between the two as a potential split
                
                #consider all values which lie between two values as a potential split
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

In [48]:
def split_data(X, y, split_column, split_value):
    
    'splits data based on specific value, will yield both a split for the features X and target y'
    
    split_column_values = X[:, split_column]

    X_below = X[split_column_values <= split_value] #partitions data according to split values from previous functions
    X_above = X[split_column_values >  split_value]
    
    y_below = y[split_column_values <= split_value]
    y_above = y[split_column_values >  split_value]
    
    return X_below, X_above, y_below, y_above

In [49]:
def calculate_entropy(y):
    
    'calculates entropy for each partition of data'
    
    _, counts = np.unique(y, return_counts=True) #we only need the counts of each class, _ is a placeholder

    probabilities = counts / counts.sum() #probability of each class
    entropy = sum(probabilities * -np.log2(probabilities)) #could replace with Gini impurity or misclassification
     
    return entropy

In [50]:
def calculate_overall_entropy(y_below, y_above): 
    
    'calculates the total entropy after each split'
       
    n = len(y_below) + len(y_above)
    p_data_below = len(y_below) / n
    p_data_above = len(y_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(y_below)
                      + p_data_above * calculate_entropy(y_above))
    
    return overall_entropy

In [51]:
def determine_best_split(X, y, potential_splits):
    
    'selects which split lowered entropy the most'
    
    overall_entropy = 9999 #set arbitrarily high, the function will loop over and replace this with lower impurity values
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            X_below, X_above, y_below, y_above = split_data(X, y, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(y_below, y_above)
            
            #goes through each potential split and only updates if it lowers entropy

            if current_overall_entropy <= overall_entropy: 
                overall_entropy = current_overall_entropy #updates only if lower entropy split found, in the ned this is greedy search
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

### The Main Algorithm

The tree will now implement the helper functions and display the decision which yielded the best split if printed. Since the growing the tree can take a little bit of time, the argument `verbose` also exists which gives you status updates on the algorithm's progress. This is common for quite a few functions in the programming world.

In [52]:
def decision_tree_algorithm(X, y, counter=0, min_samples=2, max_depth=5, verbose=False): 
    
    # data preparation
    if counter == 0:  # counter tells us how deep the tree is, this is before the tree is initiated
        global COLUMN_HEADERS
        COLUMN_HEADERS = X.columns
        X = X.values  # change all to NumPy array for faster calculations
        y = y.values
        if verbose:
          print("Data preparation complete, tree initiated")
          print("Creating first branch until first leaf, branches will then be calculated sequentially")
    else:
        data = X  # if we have started the tree, X should already be a NumPy array from the code above 
    # base cases
    if (check_purity(y)) or (len(X) < min_samples) or (counter == max_depth):
        if verbose:
          if (check_purity(y)):
              print("Classes pure, new leaf node created")
          elif (len(X) < min_samples):
              print("Min samples reached, new leaf node created")
          else:
              print("Max depth reached, new leaf node created")
        classification, prob = classify_data(y)
        return classification, prob
    
    # recursive part
    else:    

        # helper functions 
        if verbose:
            print("Calculating splits at depth of", counter)
        potential_splits = get_potential_splits(X)  # check for all possible splits

        if bool([feature_splits for feature_splits in potential_splits.values() if feature_splits != []]): # Checks that potential_splits is not just a blank dictionary (occurs when random feature subspace has no variation in observations)        
            best_split_column, best_split_value = determine_best_split(X, y, potential_splits)  # select best split based on entropy
            X_below, X_above, y_below, y_above = split_data(X, y, best_split_column, best_split_value)  # execute best split
         
            # Code to explain decisions made by tree to users
            feature_name = COLUMN_HEADERS[best_split_column]
            question = "{} <= {}".format(feature_name, best_split_value)  # initiate explanation of split
            sub_tree = {question: []}
        
            # Pull answers from tree
            if verbose:
                print("Processing first split at depth", counter)
            yes_answer = decision_tree_algorithm(X_below, y_below, counter + 1, min_samples, max_depth, verbose)
            if verbose:
                print("Processing second split at depth", counter)
            no_answer = decision_tree_algorithm(X_above, y_above, counter + 1, min_samples, max_depth, verbose)
        
            # Ensure explanation actually shows useful information
            if yes_answer == no_answer:  # if decisions are the same, only display one
                sub_tree = yes_answer
            else:
                sub_tree[question].append(yes_answer)
                sub_tree[question].append(no_answer)

            if verbose:
             if (counter == 0):
                print("Tree Complete")

        else: # if potential_splits is empty due to lack of feature variation, just classify the data at this point
            classification = classify_data(y)
            return classification
        
        return sub_tree

### Grow First Tree

Test the tree and display the decisions with a shallow depth.

In [53]:
tree = decision_tree_algorithm(X, y, max_depth=2, verbose=True)

Data preparation complete, tree initiated
Creating first branch until first leaf, branches will then be calculated sequentially
Calculating splits at depth of 0
Processing first split at depth 0
Calculating splits at depth of 1
Processing first split at depth 1
Max depth reached, new leaf node created
Processing second split at depth 1
Max depth reached, new leaf node created
Processing second split at depth 0
Calculating splits at depth of 1
Processing first split at depth 1
Max depth reached, new leaf node created
Processing second split at depth 1
Max depth reached, new leaf node created
Tree Complete


#### Structure of the First Tree

In [54]:
from pprint import pprint  # library for prettier print output

pprint(tree)

{'DEBTINC <= 0.132537517644646': [{'DELINQcat_1+ <= 0.5': [(False,
                                                            0.04424379232505643),
                                                           (False,
                                                            0.2777777777777778)]},
                                  {'DEBTINC <= 0.137471043429223': [(True,
                                                                     0.6182246661429693),
                                                                    (False,
                                                                     0.11466325660699062)]}]}


In [55]:
tree.values()

dict_values([[{'DELINQcat_1+ <= 0.5': [(False, 0.04424379232505643), (False, 0.2777777777777778)]}, {'DEBTINC <= 0.137471043429223': [(True, 0.6182246661429693), (False, 0.11466325660699062)]}]])

The first node in the tree asks the question:

DEBTINC <= 0.13253751764464605

If this is true, the tree will predict BAD as false.

If this is false, the tree will ask a second question:

DEBTINC <= 0.137471043429223

If true, the tree will predict BAD is true. If false, the tree will predict BAD is false.

#### Classification Example from First Tree

Let's take an example and see how the tree classifies it.

In [56]:
example = df.iloc[2] #take a random observation
example

BAD                     True
LOAN               -1.789048
MORTDUE            -1.654549
VALUE              -1.839275
YOJ                -0.668103
CLAGE              -0.368769
NINQ               -0.065054
CLNO               -1.189302
DEBTINC             0.137456
DEROGzero               True
REASON_HomeImp             1
REASON_IsMissing           0
JOB_Office                 0
JOB_Other                  1
JOB_ProfExe                0
JOB_Sales                  0
JOB_Self                   0
DELINQcat_1                0
DELINQcat_1+               0
Name: 2, dtype: object

To put this example down our tree, we implement a function `classify_example()`. Eventually, we would like our tree to facilitate prediction in a similar way as `sklearn` models. Therefore, we also implement two *wrapper* functions `predict()` and `predict_proba()`; as we did before for our custom logit model. These functions kind of 'wrap up' the function `classify_example()` - hence the name wrapper function - and call it for every instance in a data set.

In [57]:
def classify_example(example, tree):
    question = list(tree.keys())[0] #checks what is the next data split
    feature_name, comparison_operator, value = question.split() #splits question into elements

    # ask question
    if example[feature_name] <= float(value): # checks key element for split
        answer = tree[question][0] # selects yes answer
    else:
        answer = tree[question][1] # selects no answer

    # base case
    if not isinstance(answer, dict): # if answer is not a dictionary, we have reached a decision
        return answer # display prediction
    
    # recursive part
    else:
        residual_tree = answer # continue if another dictionary is reached
        return classify_example(example, residual_tree)

# function for predicting class of observation
def predict(tree, data):
  pred = data.apply(classify_example, axis=1, args=(tree,))
  return [int(row[0]) for row in pred]

# function for predicting probability of observation being class 1 
def predict_prob(tree, data):
  pred = data.apply(classify_example, axis=1, args=(tree,))
  return [row[1] for row in pred]

Ok, finally, here is the classification result for the above instance:

In [58]:
classify_example(example, tree)

(True, 0.6182246661429693)

In [59]:
y.iloc[2]

BAD    True
Name: 2, dtype: bool

The classification for this one instance is correct. It will be more interesting though to examine how well the tree classifiers a set of - or all - instances in our data. This is where our `predict()` functions come into play. 

#### Predictions and Accuracy from First Tree

In [60]:
predictions = predict(tree, X)  # obtain discrete class predictions (i.e., majority class in the respective leaf nodes)

predictions[:5]

[1, 1, 1, 1, 1]

In [61]:
predictions_prob = predict_prob(tree, X) #probability of being class 1

predictions_prob[:5]

[0.6182246661429693,
 0.6182246661429693,
 0.6182246661429693,
 0.6182246661429693,
 0.6182246661429693]

In [62]:
error = np.mean(np.vstack(predictions) == np.array(y))

error

0.851006711409396

The tree is able to classify a good proportion of the test set well even with only a depth of 2.

### Higher Depth in Tree

Test the tree and display the decisions with a higher depth to see if accuracy changed. Be warned, the computations might take a few minutes to complete. After growing the tree, we will proceed as before with calculating predictions and examining their accuracy.

In [63]:
tree_deep = decision_tree_algorithm(X, y, max_depth=5, verbose=True)

Data preparation complete, tree initiated
Creating first branch until first leaf, branches will then be calculated sequentially
Calculating splits at depth of 0
Processing first split at depth 0
Calculating splits at depth of 1
Processing first split at depth 1
Calculating splits at depth of 2
Processing first split at depth 2
Calculating splits at depth of 3
Processing first split at depth 3
Calculating splits at depth of 4
Processing first split at depth 4
Max depth reached, new leaf node created
Processing second split at depth 4
Max depth reached, new leaf node created
Processing second split at depth 3
Calculating splits at depth of 4
Processing first split at depth 4
Classes pure, new leaf node created
Processing second split at depth 4
Max depth reached, new leaf node created
Processing second split at depth 2
Calculating splits at depth of 3
Processing first split at depth 3
Calculating splits at depth of 4
Processing first split at depth 4
Classes pure, new leaf node created
P

In [64]:
pprint(tree_deep)

{'DEBTINC <= 0.132537517644646': [{'DELINQcat_1+ <= 0.5': [{'YOJ <= -0.45235911313134': [{'MORTDUE <= -0.9625627627614091': [{'LOAN <= -0.5406409454191005': [(False,
                                                                                                                                                               0.46875),
                                                                                                                                                              (False,
                                                                                                                                                               0.07407407407407407)]},
                                                                                                                             {'CLAGE <= -1.6018838077330106': [(True,
                                                                                                                                                      

In [65]:
predictions_deep = predict(tree_deep, X)

predictions_deep[:5]

[1, 1, 1, 1, 1]

In [66]:
predictions_prob_deep = predict_prob(tree_deep, X)

predictions_prob_deep[:5]

[0.61003861003861,
 0.8564814814814815,
 0.61003861003861,
 0.61003861003861,
 0.61003861003861]

In [67]:
error_deep = np.mean(np.vstack(predictions_deep) == np.array(y))

error_deep

0.8897651006711409

The performance of the tree is even better with this depth. However, as we continue to increase depth, there will come a point where the accuracy decreases as the tree internalizes noise from the data. This is called overfitting and weakens the tree's ability to classify unseen observations. Thus, the tree can be deepened only until the test set accuracy begins decreasing.